# Physics 494/594

# 2D Classical Ising Model Monte Carlo

In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
π = np.pi

from IPython.display import display

## Last Time

### [Notebook Link: 27_Ising_Model_MFT.ipynb](./27_Ising_Model_MFT.ipynb)

- Implement a numerical solution for the mean field magnetization of the 2D Ising model

## Today
- Introduction to Markov Chain Monte Carlo for the Ising Model
- Generation of output configurations for machine learning

## Ising Model of Ferromagnatism

Measuring energies in units where $J=k_{\rm B} = 1$ we can write the Ising model Hamiltonian in the absence of an extenral magnetic field ($h=0$) as:

\begin{equation}
H = - \sum_{\langle i,j\rangle } \sigma_i \sigma_j
\end{equation}

where $\sigma_i = \pm 1$ and $\langle i,j \rangle$ means that sites $i$ and $j$ are nearest neighbors.  

Let's consider a $N = L \times L$ square lattice in $d=2$. We will assign $-1 = \downarrow$ and $+1 = \uparrow$.  In two dimensions, our configuration $\mathbf{x} \equiv \sigma$ is a set of spin values $\sigma_i$ which can be encoded in a matrix.

In [ ]:
def initialize_config(L):
    '''Initialize a random spin configuration'''

    σ = np.ones([L,L],dtype=int)
    σ[np.random.random([L,L])<=0.5] = -1
    return σ

In [ ]:
L = 10
σ = initialize_config(L)

# visualize
fig,ax = plt.subplots(1,2,figsize=(8,4))

L = 10
for i in range(L):
    for j in range(L):
        ax[0].annotate("", xy=(j, i+0.5*σ[i,j]), xycoords='data',
            xytext=(j, i-0.5*σ[i,j]), textcoords='data',
            arrowprops=dict(arrowstyle="-|>", color=colors[-2]))
        ax[0].scatter(i,j,c='k', s=5)

ax[0].axis('off')
ax[0].axis('equal')

ax[1].matshow(σ, cmap='binary', extent=[0,9,0,9], origin='lower');
ax[1].set_xticks([]);
ax[1].set_yticks([]);

We will consider periodic boundary conditions which is equivalent to wrapping our lattice onto the surface of a torus.

<img src="https://upload.wikimedia.org/wikipedia/commons/6/60/Torus_from_rectangle.gif" width=400px>

In terms of the matrix $\mathbf{\sigma}$, the energy can be written as:

\begin{equation}
E(\sigma) = - \frac{1}{2}\sum_{i=0}^{L-1} \sum_{j=0}^{L-1} \sigma[i,j]\left(\sigma[i+1,j] + \sigma[i-1,j] + \sigma[i,j+1] + \sigma[i,j-1]\right)
\end{equation}

How do we deal with the periodic boundary conditions?

In [ ]:
# i + 1
p1 = np.arange(1,L+1)
p1[-1] = 0

# i - 1
m1 = np.arange(-1,L-1)
m1[0] = L-1

display(p1)
display(m1)

### Thermodynamic Properties: Energy and Magnetization per Spin

In [ ]:
def get_props(σ):
    '''The energy E and magnetization M per spin for a microstate of the 2d Ising model.'''
    E,M = 0,0
    Lx = σ.shape[1]
    Ly = σ.shape[0]

    for i in range(Lx):
        for j in range(Ly):
            M += σ[i,j]
            E -= 0.5*σ[i,j]*(σ[p1[i],j] + σ[m1[i],j] + σ[i,p1[j]] + σ[i,m1[j]])
    return E/(Lx*Ly),M/(Lx*Ly)

In [ ]:
get_props(σ)

## Markov Chain Monte Carlo

If you want to learn more about Markov Chain Metropolis Monte Carlo methods see my [short course](https://github.com/agdelma/IntroMonteCarlo).

We want to sample spin configurations $\sigma$ from the Boltzmann probability:

\begin{equation}
\pi(\sigma) = \frac{\mathrm{e}^{-\beta E(\sigma)}}{\mathcal{Z}}
\end{equation}

which can be done via Metropolis Monte Carlo by constructing a Markov chain where the probability to move from configuration $\sigma$ to $\sigma'$ is given by:

\begin{equation}
\mathcal{p}(\sigma \to \sigma^\prime) = \underbrace{Q(\sigma \to \sigma^\prime)}_{\text{propose } \sigma\rightarrow \sigma^\prime \ \ \;\;\;\text{known}} \cdot \underbrace{\mathcal{A}(\sigma \to \sigma^\prime)}_{\text{accept } \sigma \rightarrow \sigma^\prime \ \ \; \text{unknown}}\, .
\end{equation}

For the Ising model, the proposal probability is simple, we just choose one spin at random with equal probability, i.e.

\begin{equation}
Q(\sigma \to \sigma^\prime) = Q(\sigma^\prime \to \sigma) = \frac{1}{N}.
\end{equation}

The solution for the acceptance probability $\mathcal{A}(\sigma \to \sigma')$ is described in one of the most important papers of the 20th century.

[N. Metropolis, A. W. Rosenbluth, M. N. Rosenbluth, A. H. Teller, and E. Teller, *Equation of State Calculations by Fast Computing Machines*, J. Chem. Phys. **21**, 1087 (1953).](https://aip.scitation.org/doi/abs/10.1063/1.1699114)

In [ ]:
import ipyplot
ipyplot.plot_images(['https://upload.wikimedia.org/wikipedia/commons/5/58/Metropolis_Nicholas_Badge.gif',
                     'https://static01.nyt.com/images/2021/02/16/obituaries/05rosenbluth2/merlin_183269562_e7e2b108-23a6-416c-ad89-eadd6e8f9e4b-mobileMasterAt3x.jpg',
                     'https://www.ias.edu/sites/default/files/styles/portrait/public/images/scholars/1798.jpg',
                     'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Edward_Teller_ID_badge.png/170px-Edward_Teller_ID_badge.png'],img_width=200,show_url=False)

Writing the detailed balance condition:

\begin{equation}
\pi(\sigma) p(\sigma \to \sigma^\prime) = \pi(\sigma^\prime) p(\sigma^\prime \to \sigma)
\end{equation}

the acceptance probability has the solution:

\begin{equation}
A(\sigma \to \sigma') = \text{min}\left[1,\frac{\pi(\sigma')}{\pi(\sigma)}\right].
\end{equation}  

**Note: this simple scheme has problems with critical slowing down (see below).**

Can be solved with cluster updates. See, e.g.: [U. Wolff,  *Collective Monte Carlo updating for spin systems*, Phys. Rev. Lett. **62**, 361 (1989).](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.62.361)

Next, we need to determine the change in energy as:

\begin{equation}
\frac{\pi(\sigma')}{\pi(\sigma)} = \frac{\mathrm{e}^{-E(\sigma')/T}}{\mathrm{e}^{-E(\sigma)/T}} = \mathrm{e}^{-\Delta E/T}
\end{equation}

where $\Delta E = E(\sigma') - E(\sigma)$.

**Note: the partition function has cancelled out of the probability!**


Suppose we flip the spin with index $k,\ell$, then: $\sigma'[k,\ell] = -\sigma[k,\ell]$ and we can write:

\begin{equation}
\Delta E = 2\sigma[k,\ell]\left(\sigma[k+1,\ell] + \sigma[k-1,\ell] + \sigma[k,\ell+1] + \sigma[k,\ell-1]\right).
\end{equation}

and

\begin{equation}
\Delta M = 2 \sigma[k,\ell]
\end{equation}

and thus the acceptance probability for the move will be:

\begin{equation}
\mathcal{A}(\sigma \to \sigma') = \min\left\{1, \mathrm{e}^{-\Delta E/T}\right\}.
\end{equation}

Each Monte Carlo update thus consists of:

1. Select a random spin to flip
2. Calculate $\Delta E$
3. Generate a uniform random number $r \in \mathcal{U}_{[0,1)}$
4. Perform the Metropolis test
    - If $r < \mathrm{e}^{-\Delta E/T}$: accept the move
    - Otherwise: do nothing
5. Measure the magnetization

A Monte Carlo **step** consists of $N=L^2$ updates.

In [ ]:
from viznet import NodeBrush, EdgeBrush
from viznet.setting import edge_setting
edge_setting['arrow_head_width'] = 0.2
edge_setting['arrow_head_length']= 0.2
empty_brush = NodeBrush('pin',ax)


def _show():
    plt.axis('off')
    plt.axis('equal')
    plt.show()

fig,ax = plt.subplots(figsize=(16,1))
brush = NodeBrush('box',ax,roundness=0.3,size=(1.0,1.0))
nodes = []
for i in range(2**3):
    nodes.append(brush >> (4*i,0))
    nodes[-1].text(f'$\sigma_{{{i}}}$')

edge_brush = EdgeBrush('->>', lw=2, color='k')
for i in range(len(nodes)-1):
    edge_brush >> (nodes[i],nodes[i+1])

_show()

In [ ]:
def monte_carlo_step(σ,T):
    '''Perform a Monte Carlo step.'''

    # get the current magnetization
    M = np.sum(σ)

    # attempt L^2 spin flips
    for update in range(σ.size):

        # get the random spin
        k = np.random.randint(0,σ.shape[0])
        ℓ = np.random.randint(0,σ.shape[1])

        # calculate the change in energy
        ΔE = 2*σ[k,ℓ]*(σ[p1[k],ℓ] + σ[m1[k],ℓ] + σ[k,p1[ℓ]] + σ[k,m1[ℓ]])

        # perform the Metropolis test
        if np.random.random() <= np.exp(-ΔE/T):
            σ[k,ℓ] *= -1

            # Update the magnetization
            M += 2*σ[k,ℓ]

    return M

### Testing it out

In [ ]:
# initialize
σ = initialize_config(L)

# visualize
plt.matshow(σ, cmap='binary', extent=[0,9,0,9]);
plt.xticks([]);
plt.yticks([]);
get_props(σ)

In [ ]:
# update
monte_carlo_step(σ,3.0)

# visualize
plt.matshow(σ, cmap='binary', extent=[0,9,0,9]);
plt.xticks([]);
plt.yticks([]);
get_props(σ)

### Monte Carlo Simulation

A simulation consists of performing a large number of Monte Carlo steps at each temperature starting from some random initial configuration.

In [ ]:
# temperatures to consider
T = np.arange(0.5,3.26,0.25)

# number of Monte Carlo steps we will perform
num_steps = 2**12

# magnetization for each temperature
M = np.zeros([num_steps,T.size])

In [ ]:
%%time
from IPython.display import clear_output,display
from time import sleep

animate = True
plot_ratio = int(num_steps / 50)

# initialize
L = 8
σ = initialize_config(L)

# create PBC lookup tables
p1 = np.arange(1,L+1)
p1[-1] = 0
m1 = np.arange(-1,L-1)
m1[0] - L-1

# Loop over temperatures from high to low
for iT,cT in enumerate(tqdm(T[::-1])):
    m = T.size - 1 - iT

    # initialize the magnetization
    M[0,m] = np.sum(σ)/L**2

    # Perform the Monte Carlo steps
    for step in range(1,num_steps):
        M[step,m] = monte_carlo_step(σ,cT)/L**2

        # we plot every plot_ratio steps
        if animate and not step % plot_ratio or step == num_steps-1:

            clear_output(wait=True)
            fig,ax = plt.subplots(ncols=1,nrows=1,figsize=(4,4))

            img = ax.matshow(σ, cmap='binary')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(f'$T = {cT:.1f}J$')

            plt.show()

### Investigate the raw data

In [ ]:
iT = np.where(T > 3)[0][0]

# compute a running average
run_ave_m = np.zeros(len(M[:,iT])-1)
for n in range(1,len(M[:,iT])-1):
    run_ave_m[n] = np.average(M[:n,iT])

# plot the raw data
plt.plot(M[:,iT],'-', label='T = %3.1f' %T[iT], alpha=0.25, color=colors[0])

# plot the running average
plt.plot(run_ave_m, linewidth=2, color=colors[0])

plt.legend(loc='upper right', frameon=True, framealpha=0.95)
plt.xlabel('Monte Carlo Step')
plt.ylabel('Magnetization per spin');

### Analyzing the results

We want to compute the average value of the magnetization at each temperature along with its standard error, but we need to skip some number of initial measurements as the system takes some *time* to **equilibrate**.

In [ ]:
skip = int(0.2*M.shape[0])
m = np.average(M[skip:],axis=0)
Δm = np.std(M[skip:],axis=0)/np.sqrt(num_steps-skip)

In [ ]:
plt.errorbar(T,np.abs(m),yerr=Δm, linewidth=1, marker='o', markersize=4, elinewidth=1, capsize=0)
plt.axvline(x=2.0/np.log(1.0+np.sqrt(2.0)), linewidth=1, color='gray', linestyle='--')
plt.xlim(0.5,3.3)
plt.ylim(-0.01,1.1)
plt.xlabel(r'Temperature ($k_{\rm B}T/J$)')
plt.ylabel('Magnetization per spin')
plt.title(f"L = {L}");

These error bars look **way** too small!  Need to perform a binning analysis!

In [ ]:
def get_binned_error(mc_data):
    '''Get the standard error in mc_data and return neighbor averaged data.'''
    N_bins = mc_data.size
    Δ = np.std(mc_data)/np.sqrt(N_bins)

    start_bin = N_bins % 2
    binned_mc_data = 0.5*(mc_data[start_bin::2]+mc_data[start_bin+1::2])

    return Δ,binned_mc_data

In [ ]:
# number of possible binning levels
num_levels = int(np.log2(M[skip:,0].size/4))+1

# compute the error at each bin level for each temperature
Δm = np.zeros([len(T),num_levels])

for iT,cT in enumerate(T):

    binned_mc_data = M[:,iT]
    for n in range(num_levels):
        Δₙ,binned_mc_data = get_binned_error(binned_mc_data)
        Δm[iT,n] = Δₙ

In [ ]:
for iT,cT in enumerate(T):
    plt.plot(np.abs(Δm[iT]),'-o', linewidth=1, markersize=5, label='$k_{\mathrm{B}}T/J = %4.2f$'%cT)
    plt.xlabel('bin level')
    plt.ylabel('Δm');
    plt.legend(ncol=1, loc=(1,0))

### Replot with improved binned error bars

In [ ]:
plt.errorbar(T,np.abs(m),yerr=np.max(Δm,axis=1), linewidth=1, marker='o', markersize=6, elinewidth=1, capsize=0)
plt.axvline(x=2.0/np.log(1.0+np.sqrt(2.0)), linewidth=1, color='gray', linestyle='--')
plt.xlim(0.5,3.3)
plt.ylim(-0.01,1.1)
plt.xlabel(r'Temperature ($k_{\rm B}T/J$)')
plt.ylabel('Magnetization per spin')
plt.title(f"L = {L}");

## Load some data for a *much* longer run and $L=32$ from disk and compare with the exact Onsager solution

\begin{equation}
m=\left[1-\left(\sinh \frac{2J}{k_{\rm B}T}\right)^{-4}\right]^{\frac {1}{8}}
\end{equation}

In [ ]:
def magnetization_exact_(T):
    '''We use units where J/k_B = 1.'''
    Tc = 2.0/np.log(1.0+np.sqrt(2.0))
    if T < Tc:
        return (1.0 - np.sinh(2.0/T)**(-4))**(1.0/8)
    else:
        return 0.0
magnetization_exact = np.vectorize(magnetization_exact_)

In [ ]:
data = np.loadtxt('../data/Ising_estimators_032.dat')
lT = np.linspace(0.01,4,1000)
lL = 32

plt.plot(lT,magnetization_exact(lT),'-', linewidth=1, label='Exact', color=colors[0])
plt.errorbar(data[:,0],np.abs(data[:,5])/lL**2,yerr=data[:,6]/lL**2, linewidth=1, marker='o',
             markersize=6, elinewidth=1, label='Monte Carlo', linestyle='None', color=colors[0])
plt.xlim(0.2,4)
plt.ylim(0,1.1)
plt.xlabel(r'Temperature ($k_{\rm B}T/J$)')
plt.ylabel('Magnetization per spin')
plt.title("L = 32")

plt.legend();

<!--
hlabel = 'm(T = %4.2f)' % T[0]
header = '%23s' % hlabel
for cT in T[1:]:
    hlabel = 'm(T = %4.2f)' % cT
    header += '%24s ' % hlabel
    
np.savetxt('Ising_magnetization_004.dat.gz',M, fmt='%24.16E', header=header)
M = np.loadtxt('Ising_magnetization_004.dat.gz')
-->